# Amazon Book Rating

# Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

# import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from transformers import AutoTokenizer

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, CountVectorizer
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

import torch 
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, pipeline

# Functions

In [2]:
def check_class_imbalance(target):
  unique, counts = np.unique(target, return_counts=True)
  plt.figure(figsize=(10, 7))
  plt.pie(counts, labels=unique, startangle=140, autopct="%1.1f%%")
  plt.legend()
  plt.show()
  
  plt.figure(figsize=(10, 7))
  sns.countplot(x=target)
  plt.xlabel("Target Class Breakdown")
  plt.ylabel("Counts")
  plt.xticks(rotation=45)
  plt.show()

def check_nulls_dupes(df):
  print(f"The Amount of Nulls: {df.isna().sum()}")
  print(f"The Amount of Dupes: {df.duplicated().sum()}")

def remove_stop_words(content):
  word_tokens = " ".join(word_tokenize(content))
  filtered_tokens = re.sub(r"[^a-zA-Z\s]", "", word_tokens).split(" ")
  filtered_word_tokens = [w.lower().strip() for w in filtered_tokens if w.lower() not in ENGLISH_STOP_WORDS]
  return " ".join(filtered_word_tokens).strip()

def lemmatize_words(content):
  
  lemma = WordNetLemmatizer()
  
  stop_free = " ".join([ word for word in content.lower().split() if word not in ENGLISH_STOP_WORDS])
  punc_free = "".join(re.sub(r"[^a-zA-Z\s]", "", stop_free))
  lemmatize_words = " ".join([lemma.lemmatize(word) for word in punc_free.split()])
  return lemmatize_words

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

def dist_plot(df):
  
  for col in df.columns:
    salary_skewness = df[col].skew()
    salary_kurtosis = df[col].kurtosis()

    plt.figure(figsize=(10, 6))
    sns.histplot(df[col], kde=True, color='blue')
    plt.title(f'Distribution of {col} with Skewness and Kurtosis')
    plt.xlabel(col)
    plt.ylabel('Frequency')

    plt.text(x=0.95, y=0.85, s=f"Skewness: {salary_skewness:.2f}", transform=plt.gca().transAxes, horizontalalignment='right')
    plt.text(x=0.95, y=0.80, s=f"Kurtosis: {salary_kurtosis:.2f}", transform=plt.gca().transAxes, horizontalalignment='right')

    plt.grid(True)
    plt.show()

# EDA

In [10]:
df = pd.read_csv("/Users/maukanmir/Downloads/archive/Books_rating.csv",nrows=50000)
df

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
...,...,...,...,...,...,...,...,...,...,...
49995,0753113880,The Other Boleyn Girl,NaN,A2FAX01N7TD75Z,"Teacher Darcey ""Darcey""",0/9,1.0,1253232000,Kindle price higher than paperback?,I'd love to read the book but I'm not paying $...
49996,0753113880,The Other Boleyn Girl,NaN,A3PVP7YJJWWIE4,Sara Rockwell,1/16,2.0,1154995200,I Already Knew the story,"I have to admit, I usually do not like histori..."
49997,0753113880,The Other Boleyn Girl,NaN,NaN,NaN,1/21,5.0,1089158400,Hahhhaaa!,"Yay! I have never read this book, but I'm read..."
49998,0753113880,The Other Boleyn Girl,NaN,A3SGLCDD8TRZI7,"Amber Waves ""doris0514""",22/25,2.0,1130544000,Badly Disappointed (spoiler at the end),I had been wanting to read this book for a lon...


# Check Nulls

In [11]:
check_nulls_dupes(df)

The Amount of Nulls: Id                        0
Title                     1
Price                 42746
User_id                9516
profileName            9517
review/helpfulness        0
review/score              0
review/time               0
review/summary            0
review/text               0
dtype: int64
The Amount of Dupes: 126


In [12]:
df.drop_duplicates(inplace=True)

In [13]:
df.drop(["Price", "User_id", "profileName", "Id"], inplace=True, axis=1)

In [14]:
df

,Title,review/helpfulness,review/score,review/time,review/summary,review/text
0,Its Only Art If Its Well Hung!,7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,Dr. Seuss: American Icon,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,Dr. Seuss: American Icon,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,Dr. Seuss: American Icon,7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,Dr. Seuss: American Icon,3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
...,...,...,...,...,...,...
49995,The Other Boleyn Girl,0/9,1.0,1253232000,Kindle price higher than paperback?,I'd love to read the book but I'm not paying $...
49996,The Other Boleyn Girl,1/16,2.0,1154995200,I Already Knew the story,"I have to admit, I usually do not like histori..."
49997,The Other Boleyn Girl,1/21,5.0,1089158400,Hahhhaaa!,"Yay! I have never read this book, but I'm read..."
49998,The Other Boleyn Girl,22/25,2.0,1130544000,Badly Disappointed (spoiler at the end),I had been wanting to read this book for a lon...


In [15]:
df.dtypes

Title                  object
review/helpfulness     object
review/score          float64
review/time             int64
review/summary         object
review/text            object
dtype: object

In [19]:
df["review/helpfulness"][0].split("/")

['7', '7']

In [20]:
def calculate_scores(value):
  
  value = value.split("/")
  first, sec = int(value[0]), int(value[-1])
  final_value = None
  try:
    final_value = first / sec
  except ZeroDivisionError as e:
    final_value = 0
  
  return final_value
    

df["review/helpfulness"] = df["review/helpfulness"].apply(calculate_scores)

In [21]:
df

,Title,review/helpfulness,review/score,review/time,review/summary,review/text
0,Its Only Art If Its Well Hung!,1.000000,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,Dr. Seuss: American Icon,1.000000,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,Dr. Seuss: American Icon,0.909091,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,Dr. Seuss: American Icon,1.000000,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,Dr. Seuss: American Icon,1.000000,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
...,...,...,...,...,...,...
49995,The Other Boleyn Girl,0.000000,1.0,1253232000,Kindle price higher than paperback?,I'd love to read the book but I'm not paying $...
49996,The Other Boleyn Girl,0.062500,2.0,1154995200,I Already Knew the story,"I have to admit, I usually do not like histori..."
49997,The Other Boleyn Girl,0.047619,5.0,1089158400,Hahhhaaa!,"Yay! I have never read this book, but I'm read..."
49998,The Other Boleyn Girl,0.880000,2.0,1130544000,Badly Disappointed (spoiler at the end),I had been wanting to read this book for a lon...
